<a href="https://colab.research.google.com/github/Avipsa1/UPPP275-Notebooks/blob/main/Making_interactive_maps_using_bokeh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making interactive maps using bokeh

<br></br>
`Bokeh` (https://bokeh.pydata.org/en/latest/) might be one of my favirites packages in python! It provides many fancy visualizations for not only spatial data. Feel free to check out some of the `bokeh` examples to get an idea what this library can accomplish.

<br></br>
In this exericse, we will learn to use `bokeh` to make an interactive map in jupyter notebook. We will be continuing using the phoneix crime data, and making an interatcive crime map. User can hover their mouses to see more information for each zipcode. 

In [ ]:
import numpy as np
!pip install geopandas;
import geopandas as gpd
from shapely.geometry import shape

from bokeh.io import output_file, show,output_notebook
from bokeh.models import ColumnDataSource,ColorBar,HoverTool
from bokeh.transform import linear_cmap
from bokeh.plotting import figure
from bokeh.palettes import Spectral6


<br></br>
Let's first of all read in the shapefile we created at the end of the Phoenix Crimte Map notebook. I also uploaded the shapefiles on Canvas, just below the Module 6 Overview. Please upload your files to Google Colab before running this command.

In [ ]:
phx_crimes = gpd.read_file("phoenix_crime_counts.shp")

<br></br>
The below command is to let notebook to display the map, very similar to `%matplotlinb inline`.


In [ ]:
output_notebook()

<br></br>
The below helper function `gdp_bokeh` is a little bit complex. It is used to convert a GeoDataFrame to the format that `bokeh` can use. Currently, `bokeh` does not support GeoDataFrame very well, so this is a work-around. But I believe, in the near future, it would be more convenient way available. At the momoent, feel free to copy-paste this helper function for your use.



In [ ]:
def gpd_bokeh(df):
    """Convert geometries from geopandas to bokeh format"""
    nan = float('nan')
    lons = []
    lats = []
    for i,shape in enumerate(df.geometry.values):
        if shape.geom_type == 'MultiPolygon':
            gx = []
            gy = []
            ng = len(shape.geoms) - 1
            for j,member in enumerate(shape.geoms):
                xy = np.array(list(member.exterior.coords))
                xs = xy[:,0].tolist()
                ys = xy[:,1].tolist()
                gx.extend(xs)
                gy.extend(ys)
                if j < ng:
                    gx.append(nan)
                    gy.append(nan)
            lons.append(gx)
            lats.append(gy)

        else:     
            xy = np.array(list(shape.exterior.coords))
            xs = xy[:,0].tolist()
            ys = xy[:,1].tolist()
            lons.append(xs)
            lats.append(ys) 

    return lons,lats

In [ ]:
phx_crimes.head()

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,Zipcode,ZIP,INC NUMBER,geometry
0,85019,85019,B5,G6350,S,9713482,16770,+33.5088785,-112.1445137,85019,85019,1439,"POLYGON ((-112.15154 33.51432, -112.15153 33.5..."
1,85020,85020,B5,G6350,S,23027500,65264,+33.5673508,-112.0535909,85020,85020,1510,"POLYGON ((-112.03919 33.59392, -112.03904 33.5..."
2,85021,85021,B5,G6350,S,17419881,74879,+33.5594209,-112.0928747,85021,85021,2372,"POLYGON ((-112.11569 33.56233, -112.11609 33.5..."
3,85022,85022,B5,G6350,S,24230483,4009,+33.6272400,-112.0484143,85022,85022,1616,"POLYGON ((-112.07416 33.64750, -112.07416 33.6..."
4,85023,85023,B5,G6350,S,18927812,18972,+33.6341180,-112.0931632,85023,85023,1806,"POLYGON ((-112.11518 33.64381, -112.11501 33.6..."


<br></br>
Now let's create a `ColumnDataSource` which is used to contain geometry as well as attributes of each zipcode. You can think about `ColumnDataSource` is `bokeh`'s DataFrame.


In [ ]:
lons, lats = gpd_bokeh(phx_crimes)
source = ColumnDataSource(data=dict(
    	x=lons, 
    	y=lats,
        zipcode = phx_crimes['Zipcode'],
        crime_counts = phx_crimes['INC NUMBER']))

<br></br>
Then, let's work on the colors for the map. Let's use the crime incidents counts to display. And we are using a color called `Spectral6` here.


In [ ]:
color_mapper = linear_cmap(field_name='crime_counts', palette=Spectral6 ,
                           low=min(phx_crimes['INC NUMBER']) ,
                           high=max(phx_crimes['INC NUMBER']))
TOOLS = "pan,wheel_zoom,reset,hover,save"

<br></br>
Here comes to how we create a map in bokeh. There are so many options, but in this example, let's create the most basic one. There are basically 4 things in the below codes.


* to create a canvas using `figure`, we can specify the size as well as give it a title.
* to create `patches`, each patch is one zipcode polygon in our case. The `x` and `y` are our geometries in the `source`.
* to create the `hover` interactivity, which allows you to hover your mouse and there would be tooltips showing up. In the tooltips, we display to attributes: 1) the crime counds and 2) the zipcode
* to add a legend color bar.






In [ ]:
map = figure(plot_width=800, plot_height=600,title="Phoenix Crime Map", tools=TOOLS,)
map.patches('x', 'y', source=source, line_color="white", line_width=0.1, color=color_mapper)

map.select_one(HoverTool).tooltips = [
    ('Crime_counts', '@crime_counts'),
    ('Zipcode', '@zipcode')
]

color_bar = ColorBar(color_mapper=color_mapper['transform'], width=16, location=(0,0))
map.add_layout(color_bar, 'right')

<br></br>
Show the map using `show` command. And let's see what happens if you hover your mouse on one of the zipcode! Really fun, right?

<br></br>
You can also export this map from notebook to a html file by using the command `output_file`.




In [ ]:
output_file("phoenix_crime.html")
show(map)